In [3]:
# Imports
from main.models import Neighborhood
from django.contrib.gis.geos import (
    GEOSGeometry, 
    Polygon,
    Point,
    MultiPolygon,
)
from pprint import pprint
from django.contrib.gis.measure import Distance

In [2]:
# Find the neighborhood that contains a given point
test_point = Point(-118.41303, 34.0658684) # Peninsula Hotel
print(Neighborhood.objects.filter(mpoly__contains=test_point))

# Check if a particular neighborhood contains a point
beverly_hills = Neighborhood.objects.get(name="Beverly Hills")
print(beverly_hills.mpoly.contains(test_point))

<QuerySet [<Neighborhood: Beverly Hills>]>
True


In [4]:
bh=beverly_hills.mpoly.centroid

In [13]:
type(bh)

django.contrib.gis.geos.point.Point

In [ ]:
# TESTING

# Define custom spatial relationship
truly_overlaps = "2********" # See https://en.wikipedia.org/wiki/DE-9IM

# For each LA zipcode, show the neighborhoods it overlaps
la_zipcodes = set([
    zipcode
    for n in Neighborhood.objects.all()
    for zipcode in Zipcode.objects.filter(mpoly__relate=(n.mpoly, truly_overlaps))
])

for zipcode in set(la_zipcodes):
    print(zipcode.zipcode, Neighborhood.objects.filter(mpoly__relate=(zipcode.mpoly, truly_overlaps)), '\n')

In [ ]:
z=Zipcode.objects.annotate(percent_water=F('water_area') / (F('water_area') + F('land_area')))
z.order_by('-percent_water')

In [ ]:
# Distance annotations
from django.contrib.gis.db.models.functions import Distance
l=Listing.objects.get(pk=287339)
d=BlockGroup.objects.annotate(distance=Distance('mpoly', l.point)).order_by('distance').first().distance

In [ ]:
# Compute the nearest ZCTA to listings 
# that are missing a zipcode region (and the distance to the boundary)
for listing in Listing.objects.filter(zipcode=None):
    zipcode = Zipcode.objects.annotate(
        distance=Distance('mpoly', listing.point)).order_by('distance').first()
    print(listing.pk, zipcode.pk, zipcode.distance)

In [ ]:
# Test: search listings by distance to POI
from django.contrib.gis.measure import D
import geocoder

poi = GEOSGeometry(geocoder.google('Beverly Hills Hotel').wkt, srid=4326)
listings = Listing.objects.filter(point__distance_lt=(poi, D(mi=0.25)))
print(listings.count())